#### sklearn.neighbors.KNeighborsClassifier
class sklearn.neighbors.KNeighborsClassifier(n_neighbors=5, *, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None)

In [1]:
import pandas as pd
credit_df = pd.read_csv('../datasets/UniversalBank.csv')
credit_df

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


In [2]:
# 성능비교를 위한 빈 데이터프레임
evaluate_df = pd.DataFrame()

In [3]:
# 변수가 아닌 컬럼 제외
data_dropped = credit_df.drop(columns = ['ID', 'ZIP Code'])

X = data_dropped.drop(columns = 'CreditCard')
y = data_dropped['CreditCard']

data_dropped.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,45,19,34,3,1.5,1,0,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,0,1


In [4]:
X.describe()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,45.338400,20.104600,73.774200,2.396400,1.937938,1.881000,56.498800,0.096000,0.104400,0.06040,0.596800
std,11.463166,11.467954,46.033729,1.147663,1.747659,0.839869,101.713802,0.294621,0.305809,0.23825,0.490589
min,23.000000,-3.000000,8.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,35.000000,10.000000,39.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,45.000000,20.000000,64.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.00000,1.000000
75%,55.000000,30.000000,98.000000,3.000000,2.500000,3.000000,101.000000,0.000000,0.000000,0.00000,1.000000
max,67.000000,43.000000,224.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.00000,1.000000


스케일링에 따라 상대적으로 가깝지만 거리가 훨씬 먼 것처럼 계산될 수 있기 때문에 분석 전 스케일링 작업 실시

근데 전체 데이터에 스케일링??

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
scaled_X = scaler.transform(X)

In [6]:
# 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size = 0.2, random_state = 42)

In [7]:
# K를 찾기 위한 하이퍼 파라미터 조정
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier()

parameters = {"n_neighbors" : [3, 5, 7, 10]
          }
gscv_knn1 = GridSearchCV (estimator = knn_clf, param_grid = parameters, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=0)
gscv_knn1.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(), n_jobs=1,
             param_grid={'n_neighbors': [3, 5, 7, 10]}, scoring='accuracy')

In [8]:
gscv_knn1.best_params_

{'n_neighbors': 10}

In [9]:
gscv_knn1.best_score_

0.7305010285554897

In [10]:
y_train_pred = gscv_knn1.predict(X_train)
y_pred = gscv_knn1.predict(X_test)

# evaluation
import sklearn.metrics as mt
train1 = [mt.accuracy_score(y_train, y_train_pred), mt.recall_score(y_train, y_train_pred),
          mt.precision_score(y_train, y_train_pred), mt.f1_score(y_train, y_train_pred)]
test1 = [mt.accuracy_score(y_test, y_pred), mt.recall_score(y_test, y_pred),
          mt.precision_score(y_test, y_pred), mt.f1_score(y_test, y_pred)]

evaluate_df['train1'] = train1
evaluate_df['test1'] = test1

print(mt.confusion_matrix(y_test, y_pred))

[[680  28]
 [234  58]]


스케일링 X

In [12]:
# 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [13]:
# K를 찾기 위한 하이퍼 파라미터 조정
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier()

parameters = {"n_neighbors" : [3, 5, 7, 10]
          }
gscv_knn2 = GridSearchCV (estimator = knn_clf, param_grid = parameters, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=0)
gscv_knn2.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(), n_jobs=1,
             param_grid={'n_neighbors': [3, 5, 7, 10]}, scoring='accuracy')

In [14]:
gscv_knn2.best_params_

{'n_neighbors': 10}

In [15]:
gscv_knn2.best_score_

0.689252523025809

In [16]:
y_train_pred = gscv_knn2.predict(X_train)
y_pred = gscv_knn2.predict(X_test)

# evaluation
import sklearn.metrics as mt
train2 = [mt.accuracy_score(y_train, y_train_pred), mt.recall_score(y_train, y_train_pred),
          mt.precision_score(y_train, y_train_pred), mt.f1_score(y_train, y_train_pred)]
test2 = [mt.accuracy_score(y_test, y_pred), mt.recall_score(y_test, y_pred),
          mt.precision_score(y_test, y_pred), mt.f1_score(y_test, y_pred)]

evaluate_df['train2'] = train2
evaluate_df['test2'] = test2

print(mt.confusion_matrix(y_test, y_pred))

[[674  34]
 [273  19]]


학습데이터만 스케일링

In [17]:
# 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [18]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train)

In [19]:
# K를 찾기 위한 하이퍼 파라미터 조정
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier()

parameters = {"n_neighbors" : [3, 5, 7, 10]
          }
gscv_knn3 = GridSearchCV (estimator = knn_clf, param_grid = parameters, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=0)
gscv_knn3.fit(scaled_X_train, y_train)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(), n_jobs=1,
             param_grid={'n_neighbors': [3, 5, 7, 10]}, scoring='accuracy')

In [20]:
gscv_knn3.best_params_

{'n_neighbors': 10}

In [21]:
gscv_knn3.best_score_

0.7307509036179584

In [22]:
y_train_pred = gscv_knn3.predict(scaled_X_train)
y_pred = gscv_knn3.predict(X_test)

# evaluation
import sklearn.metrics as mt
train3 = [mt.accuracy_score(y_train, y_train_pred), mt.recall_score(y_train, y_train_pred),
          mt.precision_score(y_train, y_train_pred), mt.f1_score(y_train, y_train_pred)]
test3 = [mt.accuracy_score(y_test, y_pred), mt.recall_score(y_test, y_pred),
          mt.precision_score(y_test, y_pred), mt.f1_score(y_test, y_pred)]

evaluate_df['train3'] = train3
evaluate_df['test3'] = test3

print(mt.confusion_matrix(y_test, y_pred))

c:\Users\Master\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


[[708   0]
 [292   0]]


c:\Users\Master\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
evaluate_df.index = ['accuracy','precision','recall','f1-score']
evaluate_df

,train1,test1,train2,test2,train3,test3
accuracy,0.755000,0.738000,0.713500,0.693000,0.754500,0.708
precision,0.241087,0.198630,0.084890,0.065068,0.239389,0.000
recall,0.767568,0.674419,0.595238,0.358491,0.766304,0.000
f1-score,0.366925,0.306878,0.148588,0.110145,0.364812,0.000


## 결론
- 1번 데이터셋(전체 데이터에 정규화 이후 분할) 다시 말해, KNN에서 전체 데이터셋을 정규화 이후 train, test split을 하는 것이 가장 성능 좋다.
- 3번 데이터셋(분할 이후 정규화) 다시 말해, train set에서만 정규화 스케일링 이후 test set은 스케일링 하지 않을 경우 정규화된 학습 데이터셋에 매우 과적합